# Tavily Web Search
- LLM에게 웹검색을 쥐어주자

In [4]:
%pip install -q langchain-tavily

Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip is available: 24.3.1 -> 25.2
[notice] To update, run: python.exe -m pip install --upgrade pip


In [8]:
from dotenv import load_dotenv
from pprint import pprint

load_dotenv()

True

In [6]:
from langchain_tavily import TavilySearch   

search_tool = TavilySearch(
    max_results=5,
    topic="general",
    search_depth="basic",
    # time_range="day",
    # include_domains=None,
    # exclude_domains=None
)

In [9]:
pprint(search_tool.invoke({'query': '강남 신세계에서 밥먹을만한 곳 알려줘'}))

{'answer': None,
 'follow_up_questions': None,
 'images': [],
 'query': '강남 신세계에서 밥먹을만한 곳 알려줘',
 'request_id': '694d0b13-1f47-4642-89a5-21fb8b163b47',
 'response_time': 1.27,
 'results': [{'content': '강남신세계 혼밥맛집 (106곳) · 1. 소녀방앗간 서울고속터미널점 · 2. 타코벨 '
                         '서울고속버스터미널점 · 3. 소이연남 파미에스테이션점 · 4. 갓덴스시 센트럴시티점 · 5. '
                         '101번지',
              'raw_content': None,
              'score': 0.72231215,
              'title': "'강남신세계 혼밥' 맛집 빅데이터 추천순위 Top100 - 다이닝코드",
              'url': 'https://www.diningcode.com/list.dc?query=%EA%B0%95%EB%82%A8%EC%8B%A0%EC%84%B8%EA%B3%84%20%ED%98%BC%EB%B0%A5'},
             {'content': 'Missing: 밥 알려줘',
              'raw_content': None,
              'score': 0.71948266,
              'title': '강남 신세계 푸드코트 하우스오브 신세계,스위트파크(Eng CC)/ Seoul ...',
              'url': 'https://www.youtube.com/watch?v=wRvUnu8OwWI'},
             {'content': 'Missing: 밥 알려줘',
              'raw_content': None,
              'score': 0.63316137,


In [18]:
from langchain.prompts import PromptTemplate
from langchain_openai import ChatOpenAI
from langchain_core.output_parsers import StrOutputParser
from langchain_core.runnables import RunnableLambda, RunnablePassthrough

llm = ChatOpenAI(model='gpt-4.1-nano')

prompt = PromptTemplate(
    input_variables=['query', 'search_results'],
    template="""넌 웹 검색결과를 요약해서 사용자의 질문에 맞게 좋은 답을 해주는 챗봇이야.

    질문: {query}
    검색결과: {search_results}
    질문에 맞는 답을 검색결과 바탕으로 간단하게 알려줘
    """
)

chain = (
    {
        'query': RunnablePassthrough(),  # 사용자 입력 그대로 이 자리에 들어옴
        'search_results': search_tool | RunnableLambda(
            lambda x: '\n'.join(
                [f'-{r['title']} ({r['content']})' for r in x['results']]
            )
        )
    }
    | prompt
    | llm
    | StrOutputParser()
)

chain.invoke('강남 신세계에서 점심 먹기 좋은곳')

"강남 신세계백화점에서 점심 식사로 추천할 만한 곳으로는 타치바나(지하 1층 하우스오브신세계)가 있으며, 인기 맛집으로는 '포비 강남점', '베통', '김수사' 등이 있습니다. 신세계백화점 내부에서는 미쉐린 빕 구르망 셰프의 점심 한정 메뉴도 경험해보실 수 있습니다."

In [17]:
dumb_chain = prompt | llm | StrOutputParser()

dumb_chain.invoke({'query': '강남 신세계에서 점심먹기 좋은곳', 'search_results': ''})

'강남 신세계 내에서 점심하기 좋은 곳으로는 다양한 레스토랑이 있습니다. 특히, 유명한 일식, 한식, 양식 레스토랑들이 있어 선택의 폭이 넓습니다. 추천드릴 만한 곳은 신세계백화점 내부의 푸드코트와 레스토랑가에서 맛있는 점심 식사를 하실 수 있습니다.'